# Ship Classification and Object Detection

In [2]:
pwd()

'C:\\Users\\akash\\Documents\\MLProjects\\MLproject1\\notebooks'

In [3]:
cd ../

C:\Users\akash\Documents\MLProjects\MLproject1


In [4]:
import numpy as np
import cv2

In [5]:
import torch
from torchvision.models.detection import retinanet_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image

In [6]:
import os

In [7]:
path_training_images = 'C:\\Users\\akash\\Documents\\MLProjects\\ships-aerial-images\\train\\images'
path_valid_images = 'C:\\Users\\akash\\Documents\\MLProjects\\ships-aerial-images\\valid\\images'

In [8]:
files = os.listdir(path_training_images)
files = os.listdir(path_training_images)

### Load in Images

In [9]:
def image_loader(n: int = 50):
    files = os.listdir(path_training_images)
    return [Image.open(path_training_images + '\\' + i) for i in files[0:n]]

def image_loader_val(n: int = 50):
    files = os.listdir(path_training_images)
    return [Image.open(path_training_images + '\\' + i) for i in files[0:n]]

def image_tensor(images):
    return [F.to_tensor(i) for i in images]

def image_normalize(tensors):
    return [F.normalize(i,mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) for i in tensors]


In [9]:
images = image_loader(50)
image_tensors = image_tensor(images)
images_normed_train = image_normalize(image_tensors)

In [10]:
images = image_loader_val(50)
image_tensors = image_tensor(images)
images_normed_val = image_normalize(image_tensors)

### Temp use COCO images

In [11]:
pip install fiftyone

^C
Note: you may need to restart the kernel to use updated packages.


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\akash\\miniconda3\\envs\\ships\\Lib\\site-packages\\kaleido\\executable\\bin\\kaleido.exe'
Consider using the `--user` option or check the permissions.



In [ ]:
dataset = fiftyone.zoo.load_zoo_dataset("coco-2017")

In [17]:
model = retinanet_resnet50_fpn()
model

RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(256, e

In [28]:
F.to_tensor(np.array(images_normed_train))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (50, 3) + inhomogeneous part.

In [34]:
[i.shape for i in images_normed_train]

[torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 768, 768]),
 torch.Size([3, 80, 80]),
 torch.Size([3, 80, 80

In [19]:
def run_train_iter(self, x, y):
        
        self.train()  # sets model to training mode (in case batch normalization or other methods have different procedures for training and evaluation)
        x, y = x.float().to(device=self.device), y.long().to(
            device=self.device)  # send data to device as torch tensors
        out = self.model.forward(x)  # forward the data in the model

        loss = F.cross_entropy(input=out, target=y)  # compute loss

        self.optimizer.zero_grad()  # set all weight grads from previous training iters to 0
        loss.backward()  # backpropagate to compute gradients for current iter loss
        
        self.learning_rate_scheduler.step(epoch=self.current_epoch)
        self.optimizer.step()  # update network parameters
        _, predicted = torch.max(out.data, 1)  # get argmax of predictions
        accuracy = np.mean(list(predicted.eq(y.data).cpu()))  # compute accuracy
    
        return loss.cpu().data.numpy(), accuracy

SyntaxError: incomplete input (4054622769.py, line 17)

In [ ]:
model.forward